In [1]:
import pandas as pd
import os
import json
import plotly.express as px
import plotly

In [2]:
success_df = pd.DataFrame(columns = ['Agent', 'Trajectory', 'Time'])

In [3]:
directory = './results/agent_1/'

# iterate through all result json files
for file_name in os.listdir(directory):
    f = open(directory + file_name)

    # account for json coding errs
    try:
        result = json.load(f)

        if result['agent_1_nn']['success'] and result['agent_1_cnn']['success']:
            # first load agent 1 results
            success_df = success_df.append([{'Agent': '1_norm', 'Trajectory': result['agent_1']['trajectory_length'], 'Time': result['agent_1']['completion_time']}])
            
            # now neural network results
            success_df = success_df.append([{'Agent': '1_nn', 'Trajectory': result['agent_1_nn']['trajectory_length'], 'Time': result['agent_1_nn']['completion_time']}])
            success_df = success_df.append([{'Agent': '1_cnn', 'Trajectory': result['agent_1_cnn']['trajectory_length'], 'Time': result['agent_1_cnn']['completion_time']}])
    
    except ValueError:
        print(file_name)
    
    # Close file socket
    f.close()

success_df = success_df.reset_index()
success_df.head()

1639552032.json


,index,Agent,Trajectory,Time
0,0,1_norm,251,0.238484
1,0,1_nn,150,4.430722
2,0,1_cnn,158,5.174028
3,0,1_norm,183,0.151486
4,0,1_nn,160,4.636814


In [4]:
success_df = success_df.convert_dtypes()
grouped_df = success_df.groupby(['Agent'])
avg_results_df = grouped_df.mean()
avg_results_df = avg_results_df.reset_index()
avg_results_df.head()

,Agent,index,Trajectory,Time
0,1_cnn,0.0,237.255636,7.917228
1,1_nn,0.0,265.188225,8.075922
2,1_norm,0.0,226.516962,0.194865


In [36]:
avg_results_df = avg_results_df.convert_dtypes()
bar_chart = px.bar(avg_results_df, x="Agent", y="Trajectory", title="Agent vs Average Trajectory", text='Trajectory')
bar_chart.update_traces(texttemplate='%{text:.3}', textposition='outside')
bar_chart.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
bar_chart.show()
plotly.io.write_image(bar_chart, file="./images/bar_chart_performance_agent1.png", scale=5)

In [12]:
avg_results_df = avg_results_df.convert_dtypes()
bar_chart = px.bar(avg_results_df, x="Agent", y="Time", title="Agent vs Average Time", text='Time')
bar_chart.update_traces(marker_color='firebrick')
bar_chart.update_traces(texttemplate='%{text:.3s}', textposition='outside')
bar_chart.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
bar_chart.show()
plotly.io.write_image(bar_chart, file="./images/bar_chart_time_agent1.png", scale=5)

In [31]:
fails_df = pd.DataFrame(columns = ['Agent', 'Fails', 'Total', 'Percent Failure'])

In [32]:
directory = './results/agent_1/'

nn_fails = 0
cnn_fails = 0
total = 0

# iterate through all result json files
for file_name in os.listdir(directory):
    f = open(directory + file_name)
    total += 1

    # account for json coding errs
    try:
        result = json.load(f)

        if not result['agent_1_nn']['success']:
            nn_fails += 1
        
        if not result['agent_1_cnn']['success']:
            cnn_fails += 1
    
    except ValueError:
        print(file_name)
    
    # Close file socket
    f.close()

fails_df = fails_df.append([{'Agent': '1_nn', 'Fails': nn_fails, 'Total': total}])
fails_df = fails_df.append([{'Agent': '1_cnn', 'Fails': cnn_fails, 'Total': total}])

fails_df['Percent Failure'] = fails_df['Fails'] / fails_df['Total']
fails_df['Percent Success'] = 1-fails_df['Percent Failure']

fails_df.head()

1639552032.json


,Agent,Fails,Total,Percent Failure,Percent Success
0,1_nn,228,4858,0.046933,0.953067
0,1_cnn,73,4858,0.015027,0.984973


In [35]:
fails_df = fails_df.convert_dtypes()
bar_chart = px.bar(fails_df, x="Agent", y="Percent Success", title="Agent vs Success Percent", text='Percent Success')
bar_chart.update_traces(marker_color='magenta')
bar_chart.update_traces(texttemplate='%{text:.3%}', textposition='outside')
bar_chart.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
bar_chart.show()
plotly.io.write_image(bar_chart, file="./images/bar_chart_percent_fail_agent1.png", scale=5)